In [20]:
%load_ext lab_black
import os
import pandas as pd
from dotenv import load_dotenv
from meiyume.utils import RedShiftReader, S3FileManager
import boto3
import io

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [13]:
file_manager = S3FileManager()
s3_keys = file_manager.get_matching_s3_keys(
    prefix="Feeds/BeautyTrendEngine/WebAppData/"
)
s3_keys = list(s3_keys)
len(s3_keys)

27

In [26]:
s3_keys[0]

{'Key': 'Feeds/BeautyTrendEngine/WebAppData/category_page_distinct_brands_products',
 'LastModified': datetime.datetime(2021, 1, 19, 8, 47, 8, tzinfo=tzutc()),
 'ETag': '"3126e3b9d6db656b32d7b1d8b230aff7"',
 'Size': 14410,
 'StorageClass': 'STANDARD'}

In [25]:
s3_keys_name_list = [x["Key"].split("/")[-1] for x in s3_keys]
s3_keys_name_list

['category_page_distinct_brands_products',
 'category_page_item_package_oz',
 'category_page_item_variations_price',
 'category_page_new_ingredients',
 'category_page_new_products_count',
 'category_page_new_products_details',
 'category_page_pricing_data',
 'category_page_reviews_by_user_attributes',
 'category_page_top_products',
 'ing_page_ing_data',
 'landing_page_data',
 'meta_product_launch_intensity_category_month',
 'meta_product_launch_trend_category_month',
 'meta_product_launch_trend_product_type_month',
 'new_ingredient_trend_category_month',
 'new_ingredient_trend_product_type_month',
 'prod_page_ing_data',
 'prod_page_item_data',
 'prod_page_product_review_summary',
 'prod_page_review_sentiment_influence',
 'prod_page_review_talking_points',
 'prod_page_reviews_attribute',
 'product_page_metadetail_data',
 'review_trend_by_marketing_category_month',
 'review_trend_by_marketing_product_type_month',
 'review_trend_category_month',
 'review_trend_product_type_month']

In [21]:
load_dotenv()
S3_BUCKET = "meiyume-datawarehouse-prod"
S3_PREFIX = "Feeds/BeautyTrendEngine"
S3_REGION = "ap-southeast-1"
AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")


def get_s3_client(region: str, access_key_id: str, secret_access_key: str):
    if region == "":  # or access_key_id == '' or secret_access_key == '':
        print("*ERROR: S3 client information not set*")
        return sys.exit(1)
    else:
        try:
            client = boto3.client(
                "s3",
                region,
                aws_access_key_id=access_key_id,
                aws_secret_access_key=secret_access_key,
            )
        except Exception as ex:
            client = boto3.client("s3")
    return client


def read_file_s3(
    filename: str,
    # f"{S3_PREFIX}/WebAppData",
    # "Feeds/BeautyTrendEngine/WebAppDevelopmentData/Test"
    prefix: str = f"{S3_PREFIX}/WebAppData",
    bucket: str = S3_BUCKET,
    file_type: str = "feather",
) -> pd.DataFrame:
    key = prefix + "/" + filename
    s3 = get_s3_client(S3_REGION, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
    obj = s3.get_object(Bucket=bucket, Key=key)
    if file_type == "feather":
        df = pd.read_feather(io.BytesIO(obj["Body"].read()))
    elif file_type == "pickle":
        df = pd.read_pickle(io.BytesIO(obj["Body"].read()))
    return df

In [36]:
category_page_distinct_brands_products_df = read_file_s3(
    filename="category_page_distinct_brands_products", file_type="feather"
)
json_response = category_page_distinct_brands_products_df.to_json(orient="records")

In [31]:
# category_page_distinct_brands_products_df.to_json(orient="records")
# category_page_distinct_brands_products_df.to_json(
#     "temp.json",
#     orient="records",
# )